In [4]:
import pymysql
from sqlalchemy import create_engine
import sqlalchemy
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [6]:

caps = DesiredCapabilities().CHROME
caps["pageLoadStrategy"] = "none"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [3]:
def brands_scraping(brand, wd) :

    url = 'https://www.musinsa.com/brands/%s' % brand
    wd.get(url)
    lastpage = wd.find_element(By.XPATH, '//*[@id="product_list"]/div[1]/div[1]/div[2]/div[4]/span/span[1]').text
    lastpage = int(lastpage);

    for page in (1, lastpage + 1):
        page_url = 'https://www.musinsa.com/brands/%s?category3DepthCodes=&category2DepthCodes=&category1DepthCode=&colorCodes=&startPrice=&endPrice=&exclusiveYn=&includeSoldOut=&saleGoods=&timeSale=&includeKeywords=&sortCode=NEW&tags=&page=%d&size=90&listViewType=small&campaignCode=&groupSale=&outletGoods=false&boutiqueGoods=' % (brand, page)
        for i in range(1, 91):
            # try:
            wd.get(page_url)
            print(i)
            xpath = '//*[@id="searchList"]/li[%d]/div[4]/div[2]/p[2]/a' % i
            print(xpath)
            items_url = wd.find_element(By.XPATH, xpath).get_attribute('href')
            wd.get(items_url)
            print(items_url)
            text = wd.find_element(By.XPATH, '//*[@id="page_product_detail"]/div[3]/div[3]/span/em').text
            print(text)
            # except:
            #     continue

    return

In [4]:
def musinsa_scraping():
    wd = webdriver.Chrome('chromedriver', options=chrome_options)
    wd.implicitly_wait(3)

    url = 'https://www.musinsa.com/brands'
    brands_scraping('yale', wd)

In [36]:
def cody_scraping():
    wd = webdriver.Chrome('chromedriver', options=chrome_options)
    wd.implicitly_wait(3)
    codyDf = pd.DataFrame(columns={"cody_category"})
    clothesDf = pd.DataFrame(columns=("cody_id", "clothes_maincategory", "clothes_subcategorry", "brand", "color", "season", "gender",
                                        "spring", "summer", "autumn", "winter", "img"))
    reviewDf = pd.DataFrame(columns=("clothes_id", "gender", "height", "weight", "size", "size_review", "bright", "color", "thick"))

    codyCategory = ["americancasual", "casual", "chic", "dandy", "formal", "girlish", "golf", "retro", "romantic", "sports", "street"]
    codyIdx = 0;
    clothesIdx = 0;
    for category in codyCategory:
        for i in range(1, 2): #3
            url = "https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=%s&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=view_cnt&page=%d" % (category, i)
            for j in range(1, 2): #61
                wd.get(url)
                wd.find_element(By.XPATH, "/html/body/div[3]/div[3]/form/div[4]/div/ul/li[%d]/div[1]/a" % j).click()
                codyDf.loc[codyIdx] = [category]
                codyIdx += 1
                ###################################################
                # 옷 정보 크롤링
                ###################################################
                for info in range(1, 11):
                    # try:
                    info_url = wd.find_element(By.XPATH, '//*[@id="style_info"]/div[3]/div[2]/div/div/div[1]/div[%d]/div[1]/a'%info).get_attribute('href')
                    wd.get(info_url)
                    print(wd.find_element(By.XPATH, '//*[@id="page_product_detail"]/div[3]/div[3]/span/em').text)
                    clothesMaincategory = wd.find_element(By.XPATH,'//*[@id="page_product_detail"]/div[3]/div[3]/div[1]/p/a[1]').text
                    clothesSubCategory = wd.find_element(By.XPATH, '//*[@id="page_product_detail"]/div[3]/div[3]/div[1]/p/a[2]').text
                    check = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[1]/h4').text
                    img = wd.find_element(By.XPATH, '//*[@id="bigimg"]').get_attribute('src')

                    if(check == "Product Info제품정보"):
                        brand = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[1]/ul/li[1]/p[2]/strong/a').text
                        print('brand', brand, type(brand))
                        try:
                            season = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[1]/ul/li[2]/p[2]/strong').text
                        except:
                            season = 0
                        print('season', season, type(season))
                        if(season == 0):
                            gender = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[1]/ul/li[2]/p[2]/span').text
                        else:
                            gender = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[1]/ul/li[2]/p[2]/span[2]').text
                    else:
                        print(2)
                        brand = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[2]/ul/li[1]/p[2]/strong/a').text
                        print('brand', brand ,type(brand))
                        try:
                            season = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[2]/ul/li[2]/p[2]/strong').text
                        except:
                            season = 0
                        print('season', season, type(season))
                        print(np.isnan(season))
                        if(season == 0):
                            gender = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[2]/ul/li[2]/p[2]/span').text
                        else:
                            gender = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[2]/ul/li[2]/p[2]/span[2]').text
                        print('gender', gender, type(gender))
                    # clothes_maincategory clothes_subcategorry  summer autumn spring  gender brand cody_id color season winter
                    clothesDf.loc[clothesIdx] = [(codyIdx-1), clothesMaincategory, clothesSubCategory, brand, np.nan, season, gender,
                                                 np.nan, np.nan, np.nan, np.nan, img]

                    ############################################################
                    #리뷰 크롤링
                    ############################################################
                    last = False
                    reviewIdx = 0
                    for i in range(1, 3):
                        for page in range(1,6):
                            for idx in range(1, 11):
                                # 성별, 키, 몸무게
                                user = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[1]/div/div[2]/p' % idx).text
                                # user = user.split(",")
                                print('user', user)
                                reGender = user[0]
                                height = user[1]
                                weight = user[2]
                                size = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[2]/div[2]/p/span' % idx).text
                                sizeReview = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[4]/div[2]/ul/li[1]/span % idx' % idx).text
                                bright = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[4]/div[2]/ul/li[2]/span % idx' % idx).text
                                color = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[4]/div[2]/ul/li[3]/span % idx' % idx).text
                                thick = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[4]/div[2]/ul/li[4]/span % idx' % idx).text

                                #"clothes_id", "gender", "height", "weight", "size", "size_review", "bright", "color", "thick"
                                reviewDf.loc[reviewIdx] = [clothesIdx, reGender, height, weight, size, sizeReview, bright, color, thick]
                                print(reviewDf.loc[reviewIdx])
                                reviewIdx += 1
                            pageNum = wd.find_element(By.XPATH, '//*[@id="qPage"]/div/div/a[%d]' % page + 2).text
                            if(pageNum.isdigit()):
                                wd.find_element(By.XPATH, '//*[@id="qPage"]/div/div/a[%d]' % page + 2).click()
                            else:
                                last = True
                            if(last) :
                                break
                        if(last) :
                            break
                        wd.find_element(By.XPATH)
                    clothesIdx += 1

                    wd.back()
                    # except:
                    #     break
            # print(codyDf.tail())
    return codyDf, clothesDf, reviewDf

In [38]:
cody, df = cody_scraping()

[3M 신슐레이트] 코듀로이 칼라 다운 퀄팅 자켓_Brown
2
brand  <class 'str'>
season 0 <class 'int'>
False
gender  <class 'str'>
user 


IndexError: string index out of range

1

In [63]:
df

,cody_id,clothes_maincategory,clothes_subcategorry,brand,color,season,gender,spring,summer,autumn,winter
0,0,아우터,기타 아우터,LAFUDGESTORE,NaN,0,남 여 라이프,NaN,NaN,NaN,NaN
1,0,바지,코튼 팬츠,BEARDED KID,NaN,2022 F/W,남,NaN,NaN,NaN,NaN
2,0,모자,캡/야구 모자,UNDERCONTROL,NaN,2021 ALL,남 여,NaN,NaN,NaN,NaN
3,0,주얼리,목걸이/펜던트,QUARQOR,NaN,0,,NaN,NaN,NaN,NaN
4,0,스니커즈,캔버스/단화,NOVESTA,NaN,0,남 여,NaN,NaN,NaN,NaN
5,1,아우터,트러커 재킷,DAYDARE,NaN,0,남,NaN,NaN,NaN,NaN
6,1,바지,데님 팬츠,TOFFEE,NaN,2021 ALL,남,NaN,NaN,NaN,NaN
7,1,스니커즈,캔버스/단화,VANS,NaN,ALL,남 여,NaN,NaN,NaN,NaN
8,2,아우터,겨울 싱글 코트,MUSINSA STANDARD,NaN,0,여,NaN,NaN,NaN,NaN
9,2,모자,캡/야구 모자,MLB,NaN,ALL ALL,남 여,NaN,NaN,NaN,NaN
